In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from model.baseline_model import BaselineNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [3]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_10000.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/baseline_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/baseline_model


In [4]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_10000.pkl


In [5]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [7]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [8]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, 5, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [9]:
# Load model and the optimizer
model = BaselineNonlinearModel(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 3.696e+07, Test loss: 8.012e+07, MSE(e): 3.631e+00, MSE(pi1): 2.688e+01, MSE(pi2): 1.071e+00, MSE(pi3): 3.784e+00
Epoch 100, Train loss: 1.236e+06, Test loss: 2.231e+06, MSE(e): 1.214e-01, MSE(pi1): 1.147e+00, MSE(pi2): 8.977e-02, MSE(pi3): 1.019e-01
Epoch 200, Train loss: 2.270e+04, Test loss: 6.233e+04, MSE(e): 1.978e-03, MSE(pi1): 1.688e-01, MSE(pi2): 1.190e-03, MSE(pi3): 1.227e-02
Epoch 300, Train loss: 1.132e+04, Test loss: 3.734e+04, MSE(e): 1.038e-03, MSE(pi1): 6.144e-02, MSE(pi2): 6.316e-04, MSE(pi3): 3.317e-03
Epoch 400, Train loss: 6.540e+03, Test loss: 2.286e+04, MSE(e): 6.165e-04, MSE(pi1): 2.737e-02, MSE(pi2): 3.942e-04, MSE(pi3): 1.009e-03
Epoch 500, Train loss: 9.198e+03, Test loss: 2.504e+04, MSE(e): 8.345e-04, MSE(pi1): 6.026e-02, MSE(pi2): 4.966e-04, MSE(pi3): 2.499e-03
Epoch 600, Train loss: 8.557e+03, Test loss: 4.721e+04, MSE(e): 6.898e-04, MSE(pi1): 6.981e-02, MSE(pi2): 4.637e-04, MSE(pi3): 9.609e-03
Epoch 700, 

In [10]:
# Parametros de entrenamiento
start_epoch = 9000
n_epochs = 100000

batch_size = 64
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 7.228e+02, Test loss: 3.864e+03, MSE(e): 6.469e-05, MSE(pi1): 2.773e-03, MSE(pi2): 4.879e-05, MSE(pi3): 4.822e-04
Epoch 9100, Train loss: 5.686e+02, Test loss: 4.114e+03, MSE(e): 4.940e-05, MSE(pi1): 2.670e-03, MSE(pi2): 3.909e-05, MSE(pi3): 4.789e-04
Epoch 9200, Train loss: 5.257e+02, Test loss: 8.171e+04, MSE(e): 4.514e-05, MSE(pi1): 2.610e-03, MSE(pi2): 3.617e-05, MSE(pi3): 4.817e-04
Epoch 9300, Train loss: 4.992e+02, Test loss: 4.975e+09, MSE(e): 4.252e-05, MSE(pi1): 2.582e-03, MSE(pi2): 3.458e-05, MSE(pi3): 4.821e-04
Epoch 9400, Train loss: 4.783e+02, Test loss: 4.017e+09, MSE(e): 4.045e-05, MSE(pi1): 2.568e-03, MSE(pi2): 3.334e-05, MSE(pi3): 4.812e-04
Epoch 9500, Train loss: 4.607e+02, Test loss: 3.804e+09, MSE(e): 3.870e-05, MSE(pi1): 2.561e-03, MSE(pi2): 3.225e-05, MSE(pi3): 4.800e-04
Epoch 9600, Train loss: 4.451e+02, Test loss: 3.658e+09, MSE(e): 3.716e-05, MSE(pi1): 2.557e-03, MSE(pi2): 3.127e-05, MSE(p